<a href="https://colab.research.google.com/github/Lilya-te/DS-workshop2025/blob/Checkpoint_2_hypothesis_1_v2/Checkpoint2_team1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорты библиотек

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st
from datetime import datetime

# Чтение данных

In [ ]:
df_path = "marketplace.csv"

df = pd.read_csv(df_path)
display(df.head())
df.info()

# Чистка данных

## Преобразование типов

In [ ]:
df_cleaned = df.copy()

# df_cleaned = df_cleaned.astype({'reg_dt': int, 'first_buy': int, 'first_login': dt})

df_cleaned["reg_dt"] = pd.to_datetime(df_cleaned["reg_dt"], errors="coerce")
df_cleaned["first_buy"] = pd.to_datetime(df_cleaned["first_buy"], errors="coerce")
df_cleaned["first_login"] = pd.to_datetime(df_cleaned["first_login"], errors="coerce")

df_cleaned["user_id"] = df_cleaned["user_id"].apply(lambda x: int(x.strip("user_")))
df_cleaned["browser"] = df_cleaned["browser"].apply(lambda x: int(x.strip("browser_")))

## Заполнение нулей

In [ ]:
df_cleaned["first_buy"].fillna(df_cleaned[['first_login', 'reg_dt']].max(axis=1), inplace=True)

## Склеивание дублей и индексация по user_id

In [ ]:
df_cleaned = df_cleaned.groupby(by="user_id").agg(
    {
        "platform_num": "min",
        "first_login": "min",
        "reg_dt": "min",
        "browser": "min",
        "first_buy": "min",
        "target": "mean",
        "total_buy": "sum",
        "total_return": "sum",
    }
)
df_cleaned.head()

### Вспомогательные методы

In [ ]:
def shapiro_check(dataset, col, alpha=0.05, n=3000):
    """
    Шапиро. Тест, является ли распределение СВ нормальным
    """
    checked_dataset = dataset[col] if n is None else dataset[col].sample(n=n)
    stat, pvalue = st.shapiro(checked_dataset)
    print('Шапиро')
    if pvalue > alpha:
        print(f'Данные {col} скорее всего распределены нормально\n')
    else:
        print(f'Данные {col} скорее всего распределены не нормально\n')

def kstest_check(dataset, col, alpha=0.05):
    """
    Колмогоров-Смирнов. Тест, является ли распределение СВ нормальным
    """
    arr = dataset[col]
    mu = arr.mean()
    sigma = arr.std(ddof=1)
    stat, pvalue = st.kstest(arr, 'norm', args=(mu, sigma))

    print('Колмогоров-Смирнов')

    if pvalue > alpha:
        print(f'Данные {col} скорее всего распределены нормально\n')
    else:
        print(f'Данные {col} скорее всего распределены не нормально\n')

def iqr_filter(dataset, col):
    """
    Фильтр по межквартильному размаху
    """
    medi = dataset[col].median()
    Q1, Q3 = dataset[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1

    bottom, top = medi - 1.5 * IQR, medi + 1.5 * IQR

    return dataset[(dataset[col] >= bottom) & (dataset[col] <= top)]

## Убрать выбросы по межквартильному размаху

In [ ]:
df_cleaned_iqr = pd.DataFrame(data=df_cleaned)
df_cleaned_iqr = iqr_filter(df_cleaned_iqr, 'total_buy')

shapiro_check(df_cleaned_iqr, 'total_buy')
kstest_check(df_cleaned_iqr, 'total_buy')
shapiro_check(df_cleaned_iqr, 'total_return')
kstest_check(df_cleaned_iqr, 'total_return')

## Убрать выбросы по 99-ому процентилю

In [ ]:
q99 = df_cleaned["total_buy"].quantile(0.99)
df_q99_filtered = df_cleaned[df_cleaned["total_buy"] < q99]
df_q99_filtered = df_q99_filtered.drop([383]) # ручная чистка выброса, не отсечённого процентилем

shapiro_check(df_q99_filtered, 'total_buy')
kstest_check(df_q99_filtered, 'total_buy')
shapiro_check(df_q99_filtered, 'total_return')
kstest_check(df_q99_filtered, 'total_return')

In [ ]:
print('С фильтром по межквартильному размаху')
display(df_cleaned_iqr.describe())
print('С фильтром по 99-ому процентилю')
display(df_q99_filtered.describe())

In [ ]:
st.probplot(df_q99_filtered['total_buy'], plot=plt)

plt.show()

### Остановимся на фильтре по 99-процентилю, как довольно точному, но при этом с меньшим обрезанием покупателей с большой, но реалистичной суммой покупок и возвратов.

## Проверим распределение данных по датам.

In [ ]:
df_enriched = df_q99_filtered.copy()

# Построим для этого графики попарно
fig, axes = plt.subplots(1, 3, figsize=(24, 6))

# --- Первый график ---
# Строим график зависимости дат регистрации и дат первой покупки
axes[0].scatter(df_enriched["reg_dt"], df_enriched["first_buy"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
min_date = df_enriched["reg_dt"].min()
max_date = df_enriched["first_buy"].max()
axes[0].plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

axes[0].set_title("Когда регистрировались vs Когда купили")
axes[0].set_xlabel("Дата регистрации")
axes[0].set_ylabel("Дата первой покупки")
axes[0].legend()
axes[0].grid(True)

# --- Второй график ---
# Строим график зависимости дат первого логина и дат первой покупки
axes[1].scatter(df_enriched["first_login"], df_enriched["first_buy"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
min_date = df_enriched["first_login"].min()
max_date = df_enriched["first_buy"].max()
axes[1].plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

axes[1].set_title("Когда первый логин vs Когда купили")
axes[1].set_xlabel("Дата первого логина")
axes[1].set_ylabel("Дата первой покупки")
axes[1].legend()
axes[1].grid(True)


# --- Третий график ---
# Строим график зависимости дат регистрации и дат первого логина
axes[2].scatter(df_enriched["reg_dt"], df_enriched["first_login"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
min_date = df_enriched["reg_dt"].min()
max_date = df_enriched["first_login"].max()
axes[2].plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

axes[2].set_title("Когда регистрировались vs Когда первый логин")
axes[2].set_xlabel("Дата регистрации")
axes[2].set_ylabel("Дата первого логина")
axes[2].legend()
axes[2].grid(True)


fig.autofmt_xdate()
plt.tight_layout()
plt.show()

Видим аномальные отклонения от предсказуемой красной линии, есть объёмная группа юзеров, что регестрировались на протяжении всего периода, но покупки совершали в марте-апреле 2025.   

В ходе анализа было выявлено и подтверждено, что данные с большой разницей между регистрацией и активностью дней являются техническим артефактом (сбоем БД или склеиванием двух БД).   

**Для чистоты моделирования эти записи (8 553 строки, 34.1% от выборки) были исключены.**

Дата First_buy для этой части данных, вероятней всего, была проставлена датой загрузки их загрузки в общую базу, т.к. эти пользователи зарегистривались 1-2-3 года назад, но первый логин и покупка были в марте-апреле 2025.   
В связи с невозможностью восстановить корректную дату (и по факту больших накопленных сумм покупок) оставлять эти данные опасно, они перетянут на себя всё внимание."

Исключим эти данные, сделав очистку по более 20 дней разницы между датами регистрации и покупки-логина (диапазон 20-25 визуально по графикам сверху самый предпочтительный)

In [ ]:
condition_1 = (df_q99_filtered['first_buy'] - df_q99_filtered['reg_dt']).dt.days > 20
df_q99_filtered = df_q99_filtered[~condition_1]
condition_2 = (df_q99_filtered['first_login'] - df_q99_filtered['reg_dt']).dt.days > 20
df_q99_filtered = df_q99_filtered[~condition_2]
display(df_q99_filtered.describe())

In [ ]:
df_enriched = df_q99_filtered.copy()

# Построим для этого графики попарно
fig, axes = plt.subplots(1, 3, figsize=(24, 6))

# --- Первый график ---
# Строим график зависимости дат регистрации и дат первой покупки
axes[0].scatter(df_enriched["reg_dt"], df_enriched["first_buy"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
min_date = df_enriched["reg_dt"].min()
max_date = df_enriched["first_buy"].max()
axes[0].plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

axes[0].set_title("Когда регистрировались vs Когда купили")
axes[0].set_xlabel("Дата регистрации")
axes[0].set_ylabel("Дата первой покупки")
axes[0].legend()
axes[0].grid(True)

# --- Второй график ---
# Строим график зависимости дат первого логина и дат первой покупки
axes[1].scatter(df_enriched["first_login"], df_enriched["first_buy"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
min_date = df_enriched["first_login"].min()
max_date = df_enriched["first_buy"].max()
axes[1].plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

axes[1].set_title("Когда первый логин vs Когда купили")
axes[1].set_xlabel("Дата первого логина")
axes[1].set_ylabel("Дата первой покупки")
axes[1].legend()
axes[1].grid(True)


# --- Третий график ---
# Строим график зависимости дат регистрации и дат первого логина
axes[2].scatter(df_enriched["reg_dt"], df_enriched["first_login"], alpha=0.5, s=10)

# Добавляем линию "мгновенной покупки" (где X=Y), чтобы видеть задержку
min_date = df_enriched["reg_dt"].min()
max_date = df_enriched["first_login"].max()
axes[2].plot(
    [min_date, max_date],
    [min_date, max_date],
    color="red",
    linestyle="--",
    label="Моментальная покупка",
)

axes[2].set_title("Когда регистрировались vs Когда первый логин")
axes[2].set_xlabel("Дата регистрации")
axes[2].set_ylabel("Дата первого логина")
axes[2].legend()
axes[2].grid(True)


fig.autofmt_xdate()
plt.tight_layout()
plt.show()

Теперь хорошо.

## Выводы

Данные распределены не нормально на уровне доверия 95%. При проверке гипотез будем использовать непараметрические тесты и очистку по 99 процентилю.

# Гипотезы

### Рассмотрим следующие гипотезы:
 1. Влияние заранее зарегистрировавшихся пользователей (задолго до первого логина/покупки) и пользователей, которые произвели покупку до регистрации
 2. Влияние браузера на прибыльность или на скорость покупки.
 3. Рассмотреть превалирирование определённого браузера/браузеров у юзеров, совершивших больше всего возвратов.
 4. Мульти-платформенные аномалии

Стратегия проверки: Формулируем каждую гипотезу в формате $H_0H_1$. В дальнейшем проверяем каждую гипотезу отдельно уровне доверия 95%.


### Гипотеза №1.1 "Пришел, увидел, подумал, еще подумал, купил"
_Мы предполагаем, что пользователи, принимающие решение о покупке с разной скоростью (период времени между регистрацией и первой покупкой), приносят разную прибыль._   

### Переформулируем гипотезу в формате H₀H₁:

H₀: Total_buy не различается у групп «быстрых» и «медленных» покупателей. (Медианы групп равны).

H₁: Total_buy различается в зависимости от скорости принятия решения о покупке.

In [ ]:
df_enriched_1 = df_q99_filtered.copy()
# 1. Считаем время до покупки в днях
df_enriched_1['time_to_buy'] = (df_enriched_1['first_buy'] - df_enriched_1['reg_dt']).dt.days

# 2. Находим медиану — это наш порог разделения
threshold = df_enriched_1['time_to_buy'].median()
print(f"Медианное время до покупки: {threshold} дней")

# 3. Присваиваем названия группам
df_enriched_1['buy_speed_group'] = df_enriched_1['time_to_buy'].apply(
    lambda x: 'fast_buyers' if x <= threshold else 'slow_buyers'
)

# Проверяем размеры групп (должны быть примерно равны)
print(df_enriched_1['buy_speed_group'].value_counts())


Проверим распределение целевой переменной total_buy (прибыль) на нормальность визуально (гистограмма) и с помощью теста Колмогорова-Смирнова.

In [ ]:
def kstest_check(dataset, col, alpha=0.05):
    """
    Колмогоров-Смирнов. Тест, является ли распределение СВ нормальным
    """
    arr = dataset[col]
    mu = arr.mean()
    sigma = arr.std(ddof=1)
    stat, pvalue = st.kstest(arr, 'norm', args=(mu, sigma))

    # Убрали принт внутри, добавили возврат значений
    return stat, pvalue

# --- ВИЗУАЛИЗАЦИЯ ---
plt.figure(figsize=(10, 6))
sns.histplot(data=df_enriched_1, x='total_buy', hue='buy_speed_group', kde=True, bins=50)
plt.title('Распределение прибыли (total_buy) по группам')
plt.show()

# --- ПРОВЕРКА ПО ГРУППАМ ---
groups = ['fast_buyers', 'slow_buyers']

print('Результаты теста Колмогорова-Смирнова:\n' + '-'*40)

for group in groups:
    # Фильтруем данные для текущей группы
    group_df = df_enriched_1[df_enriched_1['buy_speed_group'] == group]

    # Теперь функция возвращает значения, и ошибка уйдет
    stat, p_value = kstest_check(group_df, 'total_buy')

    print(f"Группа {group} (n={len(group_df)}): статистика={stat:.3f}, p-value={p_value:.5e}")

    if p_value < 0.05:
        print(f"-> Распределение в группе {group} НЕ является нормальным.\n")
    else:
        print(f"-> Распределение в группе {group} нормальное.\n")

## Обоснование выбора критерия:


- Тип переменной: Количественная (total_buy — сумма покупок).

- Количество групп: 2 (сравниваем Быстрых покупателей и Медленных покупателей).

- Зависимость групп: Группы независимы (разные пользователи).

- Распределение: Данные распределены не нормально (подтверждено тестом Колмогорова-Смирнова).

### Вывод: Используем непараметрический U-критерий Манна-Уитни.

In [ ]:
# Выделяем ряды данных (берем полные данные, не сэмпл!)
group_fast = df_enriched_1[df_enriched_1['buy_speed_group'] == 'fast_buyers']['total_buy']
group_slow = df_enriched_1[df_enriched_1['buy_speed_group'] == 'slow_buyers']['total_buy']

# Запускаем тест Манна-Уитни
stat, p = st.mannwhitneyu(group_fast, group_slow)

print(f"U-критерий Манна-Уитни: статистика={stat:.0f}, p-value={p}")


# Интерпретация результатов
alpha = 0.05

if p < alpha:
    print("РЕЗУЛЬТАТ: Нулевая гипотеза (H0) ОТВЕРГАЕТСЯ.")
    print("Разница в доходах между группами статистически ЗНАЧИМА.\n")

    # Сравним медианы, чтобы понять направление различий
    median_fast = group_fast.median()
    median_slow = group_slow.median()

    print(f"Медианная прибыль (Быстрые покупатели): {median_fast:.2f}")
    print(f"Медианная прибыль (Медленные покупатели): {median_slow:.2f}")

    diff_percent = ((median_fast - median_slow) / median_slow) * 100 if median_slow != 0 else 0


    if median_fast > median_slow:
        print(f"ВЫВОД: Пользователи, покупающие БЫСТРО, приносят БОЛЬШЕ денег.")
        print(f"Разница в медианах: +{diff_percent:.1f}% в пользу быстрых.")
    else:
        print(f"ВЫВОД: Пользователи, которые долго решаются на первую покупку, приносят БОЛЬШЕ денег.")
        print(f"Разница в медианах: +{abs(diff_percent):.1f}% в пользу медленных.")

else:
    print("РЕЗУЛЬТАТ: Не удалось отвергнуть нулевую гипотезу (H0).")
    print("Статистически значимых различий в прибыли между группами НЕ обнаружено.")

Используя непараметрический U-критерий Манна-Уитни для независимых выборок, мы получили следующие результаты:

- P-value: $p < 0.05$.

- Результат: Нулевая гипотеза ($H_0$) отвергается. Различия между группами статистически значимы и не являются случайными.

Быстрые покупатели приносят на 7,9% больше прибыли.

Давайте в качестве развлечения проверим аналогичную гипотезу, но с total_return:
Мы уже разделили пользователей на быстрых и медленных покупателей в прошлом шаге, так что датасет уже готов.

# Гипотеза №1.2: "Я щас как возьму, а потом как верну"


Формулировка:

$H_0$: Объем возвратов (total_return) не отличается между быстрыми и медленными покупателями.

$H_1$: Объем возвратов различается между быстрыми и медленными покупателями.

In [ ]:
# Визуализация возвратов (total_return)
plt.figure(figsize=(10, 6))
# Ограничим ось X, так как возвратов обычно меньше, чем покупок, и могут быть выбросы
sns.histplot(data=df_enriched_1, x='total_return', hue='buy_speed_group', kde=True, bins=50)
plt.title('Распределение возвратов (total_return) по группам')
plt.show()

# Проверка на нормальность (Колмогоров-Смирнов)
groups = ['fast_buyers', 'slow_buyers']
print('Результаты теста Колмогорова-Смирнова (total_return):\n' + '-'*40)

for group in groups:
    group_df = df_enriched_1[df_enriched_1['buy_speed_group'] == group]
    stat, p_value = kstest_check(group_df, 'total_return')

    print(f"Группа {group}: статистика={stat:.3f}, p-value={p_value:.5e}")
    if p_value < 0.05:
        print(f"Распределение НЕ нормальное.")
    else:
        print(f"Распределение нормальное.")
print("-" * 40 + "\n")

# Тест Манна-Уитни (сравнение групп)
group_fast = df_enriched_1[df_enriched_1['buy_speed_group'] == 'fast_buyers']['total_return']
group_slow = df_enriched_1[df_enriched_1['buy_speed_group'] == 'slow_buyers']['total_return']

stat, p = st.mannwhitneyu(group_fast, group_slow)

print(f"U-критерий Манна-Уитни: статистика={stat:.0f}, p-value={p}")
print("-" * 40)

if p < 0.05:
    print("РЕЗУЛЬТАТ: Различия в возвратах статистически ЗНАЧИМЫ (H0 отвергается).")

    med_fast = group_fast.median()
    med_slow = group_slow.median()
    mean_fast = group_fast.mean() # Для возвратов полезно глянуть и среднее, т.к. медиана может быть 0
    mean_slow = group_slow.mean()

    print(f"\nМедиана возвратов (Fast): {med_fast:.2f} (Среднее: {mean_fast:.2f})")
    print(f"Медиана возвратов (Slow): {med_slow:.2f} (Среднее: {mean_slow:.2f})")

    if med_fast > med_slow:
        print("\n 'Быстрые' покупатели возвращают товары БОЛЬШЕ.")
    elif med_slow > med_fast:
        print("\n-'Медленные' покупатели возвращают товары БОЛЬШЕ.")
    else:
        # Если медианы равны (например, обе 0), смотрим на средние
        if mean_fast > mean_slow:
             print("\nМедианы равны, но в среднем 'Быстрые' возвращают больше.")
        else:
             print("\nМедианы равны, но в среднем 'Медленные' возвращают больше.")
else:
    print("РЕЗУЛЬТАТ: Различий в возвратах НЕТ (H0 принимается).")

Статистическое решение

P-value: $p > 0.05$.

Результат: Нулевая гипотеза принимается. Скорость принятия решения о первой покупке (быстро или медленно) не влияет на объем возвратов в будущем.



### Гипотеза №2 "Различия в выручке на пользователя, возвратах на пользователя и скорости покупки между популярными и непопулярными браузерами"

Рассмотрим популярность браузеров по количеству покупок

In [ ]:
browsers_df = (
    df_q99_filtered
    .groupby("browser")
    .agg(
        count=("browser", "count"),
    )
).sort_values("count", ascending=False)

total = browsers_df['count'].sum()
print("total count", total)

browsers_df.head(8)

Браузеры 2 и 3 выделим в отдельную группу как популярные.

In [ ]:
top_browsers = [2, 3]
df_q99_filtered_2 = df_q99_filtered.copy()
df_q99_filtered_2["first_buy_reg_diff"] = (
    df_q99_filtered_2["first_buy"] - df_q99_filtered_2["reg_dt"]
)
df_enriched_top_browsers = df_q99_filtered_2[df_q99_filtered_2['browser'].isin(top_browsers)]
df_enriched_low_browsers = df_q99_filtered_2[~df_q99_filtered_2['browser'].isin(top_browsers)]

#### 2.1. Влияние популярности браузера на выручку. Переформулируем гипотезу в формате H₀H₁:
$H_0$: Порядок значений выручки на пользователя не отличается в популярных браузерах

$H_1$: Порядок значений выручки на пользователя отличается в популярных браузерах.

Рассмотрим нормальность распределений:

In [ ]:
print('Популярные браузеры:')
shapiro_check(df_enriched_top_browsers, 'total_buy')

print('Менее популярные браузеры:')
shapiro_check(df_enriched_low_browsers, 'total_buy')

Тест Шапиро-Уилка показывает, что данные распределены не нормально.   
Прибыль — количественный показатель, групп по браузерам 2, данные в них независимы и распределены не нормально, значит для проверки гипотезы используем U-критерий Манна-Уитни:

In [ ]:
stat, p = st.mannwhitneyu(df_enriched_top_browsers['total_buy'], df_enriched_low_browsers['total_buy'])
print(stat, p)

__p-value меньше 0.05, U-критерий Манна-Уитни отвергает гипотезу $H_0$ и показывает уверенное влияние популярности браузера на выручку от пользователя__

#### 2.2. Влияние популярности браузера на возвраты. Переформулируем гипотезу в формате H₀H₁:
$H_0$: Порядок значений возвратов на пользователя не отличается в популярных браузерах

$H_1$: Порядок значений возвратов на пользователя отличается в популярных браузерах

Рассмотрим нормальность распределений:

In [ ]:
print('Популярные браузеры:')
shapiro_check(df_enriched_top_browsers, 'total_return')

print('Менее популярные браузеры:')
shapiro_check(df_enriched_low_browsers, 'total_return')

Тест Шапиро-Уилка показывает, что данные распределены не нормально.   
Прибыль — количественный показатель, групп по браузерам 2, данные в них независимы и распределены не нормально, значит для проверки гипотезы используем U-критерий Манна-Уитни:

In [ ]:
stat, p = st.mannwhitneyu(df_enriched_top_browsers['total_return'], df_enriched_low_browsers['total_return'])
print(stat, p)

__p-value меньше 0.05, U-критерий Манна-Уитни отвергает гипотезу $H_0$ показывает уверенное влияние популярности браузера на возвраты пользователя__

#### 2.3. Влияние браузера на скорость покупки. Переформулируем гипотезу в формате H₀H₁:
$H_0$: Порядок значений скорости покупки пользователями не отличается в популярных браузерах.

$H_1$: Порядок значений скорости покупки пользователями отличается в популярных браузерах.

Рассмотрим распределение скорости покупки:


In [ ]:
print('Популярные браузеры:')
shapiro_check(df_enriched_top_browsers, 'first_buy_reg_diff')

print('Менее популярные браузеры:')
shapiro_check(df_enriched_low_browsers, 'first_buy_reg_diff')

Данные распределены не нормально, аналогично предыдущему пункту используем U-критерий Манна-Уитни:

In [ ]:
stat, p = st.mannwhitneyu(df_enriched_top_browsers['first_buy_reg_diff'], df_enriched_low_browsers['first_buy_reg_diff'])
print(stat, p)

__p-value больше 0.05, U-критерий Манна-Уитни подтверждает гипотезу $H_0$ и не показывает влияния популярности браузера на скорость первой покупки__

## Гипотеза №3 "Браузерные войны!"
_Предпочтения браузера у юзеров, совершивших больше всего покупок/возвратов._

Рассмотрим корреляцию используемых браузеров юзерами.

### Переформулируем гипотезу в формате $H_0H_1$:

$H_0$: Количество покупок имеет идентичные медианы по браузерам.

$H_1$: Количество покупок имеет различные медианы по браузерам.

Выбор критерия

**Шаг 1**: Какой тип переменной сравниваем?
- Количественная (числовая)

**Шаг 2**: Сколько групп сравнивается?
- \>2 групп.

**Шаг 3**: Группы зависимы или независимы?
- Независимые группы: сравниваются разные пользователи в одно и то же время.

**Шаг 4**: Есть ли нормальность распределения?

Для количественных переменных проверяем нормальность (например, с помощью теста Шапиро-Уилка или визуально).

**Шаг 5**: Выбор критерия

Критерий Краскела-Уоллиса

In [ ]:
def kruskal_statistic(args):
    """
    Критерий Краскела-Уоллиса
    """
    h_statistic, p_value = st.kruskal(*args)

    result = "На уровне значимости 0.05 "
    if p_value < 0.05:
        result += "нулевая гипотеза отвергается. Есть значимые различия между группами."
    else:
        result += "нулевая гипотеза НЕ отвергается. Нет значимых различий между группами."
    return result

print("Группировка первых покупок по браузерам и дням")
df_enriched_3 = df_q99_filtered.copy()

df_enriched_3 = df_enriched_3.groupby(['first_buy',"browser" ]).agg(purchase_count=('total_buy', 'count'))

df_enriched_3 = df_enriched_3.reset_index()

display(df_enriched_3.head())
shapiro_check(df_enriched_3, 'purchase_count', n=None)
kstest_check(df_enriched_3, 'purchase_count')

def df_browsers_array(dataset):
    """
    Собираем количество покупок в массив для сравнения
    """
    result = []
    for browser_n in dataset["browser"].unique():
        df_browsers_n = dataset.loc[dataset["browser"] == browser_n]
        df_browsers_n = df_browsers_n.reset_index(drop=True)
        if len(df_browsers_n["browser"].unique()) > 1:
            shapiro_check(df_browsers_n, 'purchase_count', n=None)
            kstest_check(df_browsers, 'purchase_count')
        result.append(df_browsers_n['purchase_count'].values)
    return result

kruskal_statistic(df_browsers_array(df_enriched_3))

## Гипотеза №3.1
_Среднее количество покупок на популярных браузерах отличается от неполулярных._

# Переформулируем гипотезу в формате $H_0H_1$:

$H_0$: Среднее количество покупок на N популярных браузерах статистически значимо НЕ отличается от кол-ва покупок на остальных.

$H_1$: Среднее количество покупок на N популярных браузерах статистически значимо отличается от кол-ва покупок на остальных.

Выбор критерия

**Шаг 1**: Какой тип переменной сравниваем?
- Количественная (числовая)

**Шаг 2**: Сколько групп сравнивается?
- 2 группы.

**Шаг 3**: Группы зависимы или независимы?
- Независимые группы: сравниваются разные пользователи в одно и то же время.

**Шаг 4**: Есть ли нормальность распределения?
Нет

**Шаг 5**: Выбор критерия
U-критерий Манна — Уитни

In [ ]:
def mannwhitneyu_statistic(args):
    """
    Критерий Манна-Уитни
    """
    h_statistic, p_value = st.mannwhitneyu(*args)

    result = f"Для {N} браузеров на уровне значимости 0.05 "
    if p_value < 0.05:
        result += f"отвергается нулевая гипотеза. \nСреднее количество покупок на {N} популярных браузерах статистически значимо отличается от кол-ва покупок на остальных.\n\n"
    else:
        result += f"принимается нулевая гипотеза. \nСреднее количество покупок на {N} популярных браузерах статистически значимо НЕ отличается от кол-ва покупок на остальных.\n\n"
    return result

for N in [1,2,3]:
    df_enriched_3_1 = df_enriched_3.copy()
    top_browsers = df_enriched_3_1[["browser", "purchase_count"]].groupby("browser").sum().nlargest(n=N, columns="purchase_count", keep='last').index.values
    df_top_browsers = df_enriched_3_1[df_enriched_3_1["browser"].isin(top_browsers)][["first_buy", "purchase_count"]]
    df_other_browsers = df_enriched_3_1[~df_enriched_3_1["browser"].isin(top_browsers)][["first_buy", "purchase_count"]]

    df_enriched_3_1 = df_top_browsers.merge(df_other_browsers, how='outer', on="first_buy", suffixes=("_top","_other")).fillna(0).groupby("first_buy").sum()

    print(mannwhitneyu_statistic([df_enriched_3_1["purchase_count_top"].values, df_enriched_3_1["purchase_count_other"].values]))


### Вывод:

Можем заметить, что для $N = 1$ уже видим статистически значимые отличия, то есть мы имеем один браузер, значимо превышающий остальные по количеству покупок.

In [ ]:
heatmap_df = (
    df_enriched_3[df_enriched_3["browser"] < 10].pivot_table(
        index='browser',
        columns='first_buy',
        values='purchase_count',
        aggfunc='sum',
        fill_value=0
    )
)
plt.figure(figsize=(14, 6))

sns.heatmap(
    heatmap_df,
    cmap='YlOrRd',
    linewidths=0.5
)

plt.xlabel('Дата первого заказа')
plt.ylabel('Браузер')
plt.title('Тепловая карта покупок по браузерам и датам')

plt.tight_layout()
plt.show()


_По тепловой карте видим, что лидирующий браузер по количеству покупок - браузер с идентификатором **2**._

### Гипотеза №4 Этап формулирования гипотезы

Для формулирования гипотезы построим график покупок/возвратов относительно даты первого логина.    
Этот график не покажет точные даты покупок, так как общая разность Покупки и Возврата каждого клиента будет приложена к дате его первого логина.   
Но может показать клиентов, кто внёс большой вклад, с привязкой к дате его привлечения.   

In [ ]:
# Подготовка данных
df_enriched_4 = df_q99_filtered.copy()

df_enriched_4["period"] = df_enriched_4["first_buy"].dt.to_period("W")

profit_series = df_enriched_4.groupby("period")["total_buy"].sum()
loss_series = df_enriched_4.groupby("period")["total_return"].sum() * -1

plot_data = pd.DataFrame(
    {"Прибыльные": profit_series, "Убыточные": loss_series}
).fillna(0)


# График 1
ax = plot_data.plot(
    kind="bar", stacked=True, color=["green", "red"], width=0.9, figsize=(15, 6)
)
ax.set_title("Баланс прибыли и убытков (Total buy минус Total return)")
ax.set_ylabel("Сумма (Cost Value)")


plt.show()


Согласно данного графика видно, что достаточное количество людей и покупают и возвращают товары.   
Посмотрим на них в разрезе максимально "убыточных" клиентов (тех, кто возвращает товаров больше, чем покупает).   
Построим точечное распределение пользователей в зависимости от их разницы между покупками  и возвратами (введём как Cost Value).

In [ ]:
# Обогащение данных
df_enriched_4["cost_value"] = df_enriched_4["total_buy"] - df_enriched_4["total_return"]

# Построение графика
plt.figure(figsize=(10, 10))
plt.scatter(
    df_enriched_4["total_buy"],
    df_enriched_4["total_return"],
    alpha=0.4,
    s=5,
    c="blue",
    label="Пользователи",
)

# Линия безубыточности (Y = X)
# Если точка на линии, значит клиент вернул всё, что купил (Cost value = 0)
# Если выше линии - вернул больше, чем купил (Cost value < 0) - Аномалия
max_val = max(df_enriched_4["total_buy"].max(), df_enriched_4["total_return"].max())
plt.plot(
    [0, max_val],
    [0, max_val],
    color="red",
    linestyle="--",
    linewidth=2,
    label="Линия Return = Buy",
)

plt.title("Корреляция: Сумма покупок vs Сумма возвратов")
plt.xlabel("Total Buy (Сумма покупок)")
plt.ylabel("Total Return (Сумма возвратов)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)


# Клиенты, у которых возвратов больше, чем покупок или равенство
serial_returners = df_enriched_4[
    df_enriched_4["total_return"] >= df_enriched_4["total_buy"]
]

plt.scatter(
    serial_returners["total_buy"],
    serial_returners["total_return"],
    color="orange",
    s=5,
    label="Serial Returners (Return >= Buy)",
)
plt.legend()

plt.show()

# Вывод статистики по "Серийным возвращенцам"
print(
    f"Найдено {len(serial_returners)} пользователей, у которых Сумма возвратов >= Сумма покупок."
)
print("\nТоп-10 таких пользователей по сумме убытка:")
display(serial_returners.sort_values("cost_value").head(10))


Замечено любопытный момент, в Топ-10 таких пользователей довольно много имеют значительные цифры в platform_num (количество платформ, с которых они заходили. Сформулирую окончательную гипотезу.)

### Гипотеза №4 "Мульти-платформенные аномалии"
_Проверка связи между количеством использованных платформ и убыточностью клиента._

При визуальном анализе пользователей, у которых сумма возвратов превышает сумму покупок было замечено, что многие из них имеют аномально высокое количество уникальных платформ (platform_num), с которых осуществлялся вход. В то время как обычный пользователь использует 1-3 устройства (телефон, ноутбук, планшет), некоторые "убыточные" клиенты имеют десятки входов с разных платформ, что может указывать на бот-фермы.

### Сформируем гипотезу в формате $H_0H_1$:
$H_0$: Медианное количество платформ (platform_num) у убыточных клиентов (serial_returners) не отличается от медианного количества платформ у прибыльных клиентов. Различия случайны.

$H_1$: Медианное количество платформ у убыточных клиентов статистически значимо выше, чем у прибыльных.

Построение нулевой и альтернативной гипотез.   
● Выбор уровня значимости:   _0,05_   
● Сбор данных для проверки гипотезы: _Тип клиента (доходный/убыточный, если отношение возврата к общей сумме больше 30%)._  
● Выбор статистического теста.   
● Проведение статистического теста, вычисление p-value.   
● Сравнение p-value c уровнем значимости и вывод, отклонить или не отклонить нулевую гипотезу.   

Алгоритм выбора критерия   
Шаг 1: Какой тип переменной сравниваем?   
● Количественная дискретная (platform_num).

Шаг 2: Сколько групп сравнивается?   
● 2 группы.

Шаг 3: Группы зависимы или независимы?   
● Независимые группы: Данные по разным пользователям.   

Шаг 4: Есть ли нормальность распределения?   
Распределение количества платформ, скорее всего, ненормальное (скошено вправо, "длинный хвост"). Проверим это отдельно.

Выбранный метод: **U-критерий Манна-Уитни**

In [ ]:
# Сегментация пользователей
# "Серийные возвращенцы" (убыточные клиенты) возвращают более 30% товаров.
serial_returners_mask = (
    df_enriched_4["total_return"]
    / (df_enriched_4["total_buy"] + df_enriched_4["total_return"])
    > 0.3
)
group_loss = df_enriched_4[serial_returners_mask]["platform_num"]

# "Нормальные" (прибыльные)
group_profit = df_enriched_4[~serial_returners_mask]["platform_num"]

# Текстовый вывод
print(f"Количество убыточных клиентов: {len(group_loss)}")
print(f"Количество прибыльных клиентов: {len(group_profit)}")
print(f"\nМедиана платформ (Убыточные): {group_loss.median()}")
print(f"Медиана платформ (Прибыльные): {group_profit.median()}")
print(f"Среднее платформ (Убыточные):  {group_loss.mean():.2f}")
print(f"Среднее платформ (Прибыльные): {group_profit.mean():.2f}")

# Визуализация распределений
plt.figure(figsize=(10, 6))
plt.boxplot(
    [group_profit, group_loss], tick_labels=["Прибыльные", "Убыточные (Return >= Buy)"]
)
plt.title("Распределение количества платформ у прибыльных и убыточных клиентов")
plt.ylabel("Количество платформ (platform_num)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

# Проверка гипотезы (Манна-Уитни)
stat, p_value = st.mannwhitneyu(group_loss, group_profit, alternative="greater")
print(f"Статистика U: {stat}")
print(f"P-value: {p_value:.3f}")

# Вывод
alpha = 0.05
if p_value < alpha:
    print("\nВЫВОД: Отклоняем H0. Различия статистически значимы.")
    print(
        "Убыточные клиенты используют значимо больше платформ, чем нормальные пользователи."
    )
    print("Это подтверждает гипотезу о технической природе убытков.")
else:
    print(
        "\nВЫВОД: Не можем отклонить H0. Связи между убыточностью и количеством платформ не найдено."
    )


**Развернутый вывод по Гипотезе №4:**

Статистический анализ подтвердил предположение, возникшее при визуализации данных. Группа пользователей, генерирующая чистый убыток для компании, характеризуется аномальным поведением в техническом плане.

Медианное и среднее количество платформ, с которых заходили эти пользователи, статистически значимо выше, чем у добросовестных покупателей (p < 0.05).    
Наличие более "длинного хвоста" выбросов (пользователи с десятками платформ) именно в убыточном сегменте может свидетельствовать о работе автоматизированных скриптов (ботов).

**Рекомендация:**
1.  Рассмотреть user_id с platform_num > 50 отдельно. Это не похоже на поведение живых людей.
2.  Помечать аккаунт с platform_num > 10 как подозрительный.